In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Data Source

In [2]:
# Import Alphabet Soup Charity dataset
charity_df = pd.read_csv("charity_data.csv")
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Update "ASK_AMT" column with commas
charity_df["ASK_AMT"] = charity_df["ASK_AMT"].map("{:,}".format)
charity_df["ASK_AMT"]

0             5,000
1           108,590
2             5,000
3             6,692
4           142,590
            ...    
34294         5,000
34295         5,000
34296         5,000
34297         5,000
34298    36,500,179
Name: ASK_AMT, Length: 34299, dtype: object

In [4]:
# Column datatypes
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                   object
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Generate categorical variable list and count unique object values in each column
charity_df_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_df[charity_df_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
dtype: int64

# Pre-process Data

In [6]:
# Create copy of "charity_df" with selected columns
charity_df1 = charity_df[["APPLICATION_TYPE", "AFFILIATION", "USE_CASE", "ORGANIZATION", "STATUS", "SPECIAL_CONSIDERATIONS", "ASK_AMT", "IS_SUCCESSFUL"]]
charity_df1

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,N,"5,000",1
1,T3,Independent,Preservation,Co-operative,1,N,"108,590",1
2,T5,CompanySponsored,ProductDev,Association,1,N,"5,000",0
3,T3,CompanySponsored,Preservation,Trust,1,N,"6,692",1
4,T3,Independent,Heathcare,Trust,1,N,"142,590",1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,ProductDev,Association,1,N,"5,000",0
34295,T4,CompanySponsored,ProductDev,Association,1,N,"5,000",0
34296,T3,CompanySponsored,Preservation,Association,1,N,"5,000",0
34297,T5,Independent,ProductDev,Association,1,N,"5,000",1


In [7]:
# Reorder "IS_SUCCESSFUL" to first column
column1 = charity_df1.pop("IS_SUCCESSFUL")
charity_df1.insert(0, "IS_SUCCESSFUL", column1)
charity_df1

,IS_SUCCESSFUL,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT
0,1,T10,Independent,ProductDev,Association,1,N,"5,000"
1,1,T3,Independent,Preservation,Co-operative,1,N,"108,590"
2,0,T5,CompanySponsored,ProductDev,Association,1,N,"5,000"
3,1,T3,CompanySponsored,Preservation,Trust,1,N,"6,692"
4,1,T3,Independent,Heathcare,Trust,1,N,"142,590"
...,...,...,...,...,...,...,...,...
34294,0,T4,Independent,ProductDev,Association,1,N,"5,000"
34295,0,T4,CompanySponsored,ProductDev,Association,1,N,"5,000"
34296,0,T3,CompanySponsored,Preservation,Association,1,N,"5,000"
34297,1,T5,Independent,ProductDev,Association,1,N,"5,000"


In [8]:
# Create "IS_SUCCESSFUL1" column
charity_df1.insert(0, "IS_SUCCESSFUL1", charity_df1.IS_SUCCESSFUL)
charity_df1["IS_SUCCESSFUL1"].dtypes

dtype('int64')

In [9]:
# Convert "IS_SUCCESSFUL1" column to str
charity_df1["IS_SUCCESSFUL1"] = charity_df1["IS_SUCCESSFUL1"].astype(str)
charity_df1["IS_SUCCESSFUL1"].dtypes

dtype('O')

In [10]:
# Convert "IS_SUCCESSFUL1" column values
charity_df1["IS_SUCCESSFUL1"] = charity_df1["IS_SUCCESSFUL1"].replace(
    ["0", "1"],["N", "Y"]
)

charity_df1

,IS_SUCCESSFUL1,IS_SUCCESSFUL,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT
0,Y,1,T10,Independent,ProductDev,Association,1,N,"5,000"
1,Y,1,T3,Independent,Preservation,Co-operative,1,N,"108,590"
2,N,0,T5,CompanySponsored,ProductDev,Association,1,N,"5,000"
3,Y,1,T3,CompanySponsored,Preservation,Trust,1,N,"6,692"
4,Y,1,T3,Independent,Heathcare,Trust,1,N,"142,590"
...,...,...,...,...,...,...,...,...,...
34294,N,0,T4,Independent,ProductDev,Association,1,N,"5,000"
34295,N,0,T4,CompanySponsored,ProductDev,Association,1,N,"5,000"
34296,N,0,T3,CompanySponsored,Preservation,Association,1,N,"5,000"
34297,Y,1,T5,Independent,ProductDev,Association,1,N,"5,000"


In [11]:
# Generate categorical variable list and count unique object values in each column
charity_df1_cat = charity_df1.dtypes[charity_df1.dtypes == "object"].index.tolist()
charity_df1[charity_df1_cat].nunique()

IS_SUCCESSFUL1               2
APPLICATION_TYPE            17
AFFILIATION                  6
USE_CASE                     5
ORGANIZATION                 4
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
dtype: int64

In [12]:
# Create a OneHotEncoder instance
One_Hot_Encoder = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
One_Hot_Encoder_df = pd.DataFrame(One_Hot_Encoder.fit_transform(charity_df1[charity_df1_cat]))

# Add the encoded variable names to the DataFrame
One_Hot_Encoder_df.columns = One_Hot_Encoder.get_feature_names(charity_df1_cat)
One_Hot_Encoder_df

,IS_SUCCESSFUL1_N,IS_SUCCESSFUL1_Y,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,"ASK_AMT_99,879","ASK_AMT_99,982","ASK_AMT_990,776","ASK_AMT_990,952","ASK_AMT_993,707","ASK_AMT_994,372","ASK_AMT_996,155","ASK_AMT_996,860","ASK_AMT_997,073","ASK_AMT_999,468"
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
charity_df1 = charity_df1.merge(One_Hot_Encoder_df, left_index=True, right_index=True)
charity_df1 = charity_df1.drop(charity_df1_cat,1)
charity_df1

,IS_SUCCESSFUL,STATUS,IS_SUCCESSFUL1_N,IS_SUCCESSFUL1_Y,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,...,"ASK_AMT_99,879","ASK_AMT_99,982","ASK_AMT_990,776","ASK_AMT_990,952","ASK_AMT_993,707","ASK_AMT_994,372","ASK_AMT_996,155","ASK_AMT_996,860","ASK_AMT_997,073","ASK_AMT_999,468"
0,1,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34296,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34297,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Split into features and target arrays
y = charity_df1["IS_SUCCESSFUL1_Y"].values
X = charity_df1.drop(["IS_SUCCESSFUL1_Y","IS_SUCCESSFUL1_N"],1).values

# Split the preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [15]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Model

In [16]:
# Define logistic Regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train model
log_classifier.fit(X_train,y_train)

# Evaluate model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 1.000


In [17]:
# Define neural network model using Tensorflow Keras module
nn = tf.keras.models.Sequential()

# First hidden layer
num_input_features = len(X_train[0])
nn.add(tf.keras.layers.Dense(units=300, input_dim=num_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               2635200   
_________________________________________________________________
dense_1 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 2,695,601
Trainable params: 2,695,601
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile sequential model that defines loss metric and optimization function
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate model with test data using predictive accuracy
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 19s 24ms/step - loss: 0.1345 - accuracy: 0.9480
Epoch 2/50
804/804 [==============================] - 16s 20ms/step - loss: 0.0070 - accuracy: 0.9983
Epoch 3/50
804/804 [==============================] - 16s 20ms/step - loss: 0.0042 - accuracy: 0.9980
Epoch 4/50
804/804 [==============================] - 16s 20ms/step - loss: 0.0032 - accuracy: 0.9982
Epoch 5/50
804/804 [==============================] - 16s 20ms/step - loss: 0.0036 - accuracy: 0.9979
Epoch 6/50
804/804 [==============================] - 17s 21ms/step - loss: 0.0032 - accuracy: 0.9983
Epoch 7/50
804/804 [==============================] - 17s 22ms/step - loss: 0.0037 - accuracy: 0.99830s - loss: 0.0036 - accu
Epoch 8/50
804/804 [==============================] - ETA: 0s - loss: 0.0047 - accuracy: 0.99 - 17s 22ms/step - loss: 0.0047 - accuracy: 0.9981
Epoch 9/50
804/804 [==============================] - 17s 22ms/step - loss: 0.0034 - accuracy: 0.9985
Epoch 10/50
804/